In [7]:
from google.colab import drive

drive.mount('/content/gdrive', force_remount=True)

# root_path = '/content/gdrive/My Drive/Eye_in_the_sky/' # Leslie
# root_path = '/content/gdrive/My Drive/Projects/2019/Eye_in_the_sky/' # Herbert

root_path = '/content/gdrive/My Drive/Eye_in_the_sky/' # colabeyeinthesky@rug.nl

experiment = '2019-09-23 Input for Training BAG-Model'

version = 'v1'

Mounted at /content/gdrive


## Imports and settings

In particular the location of the images, labels and where to put the masks.

In [0]:
import sys
sys.path.append(root_path)

from eyeinthesky.masks import *
from eyeinthesky.shapefiles import *
from eyeinthesky.samples import get_samples

from tqdm import tqdm_notebook as progressbar

In [0]:
images_path = f'{root_path}Experiments/{experiment}/Images/'
shapes_path = f'{root_path}Experiments/{experiment}/Labels/'
masks_path = f'{root_path}Experiments/{experiment}/Masks/'
industrial_zones_path = f'{root_path}Experiments/{experiment}/Industrial zones'

industrial_zones_shape_path = f"{root_path}/industrial zones/industrial_zones_wgs1984.shp"

assert len({
    images_path, shapes_path,
    masks_path, industrial_zones_path}) == 4, (
   'Choose different locations for all, '
   'otherwise the stuff can be overwritten.')

In [10]:
samples = get_samples(
    sources=[
      ('tif', images_path),
      ('shp', shapes_path),
    ],
    destinations=[
      ('tif', masks_path),
      ('tif', industrial_zones_path),
    ]
)

print(f'Found {len(samples)} samples.')

All samples were in all folders.
Found 15 samples.


## Create pixel masks

Create masks, takes about 4 minutes for the 62 samples of approx 1GB (approx 10k x 10k x 12 channels).

In [0]:
for (
     sample,
     image_filename, label_filename,
     mask_filename, _) in progressbar(samples):

  with rasterio.open(image_filename) as dataset:
    mask_shape = (dataset.height, dataset.width)
  
  geojson = shapefile_to_pixel_geojson(image_filename, label_filename)
  mask = draw_mask(mask_shape, geojson)
  imsave(mask_filename, mask)

## Industrial zones

There are a lot industrial zones, therefore we use bounding boxes to estimate whether an image and industrial zone could overlap. We use the lat/long system, since this was used for the industrial zones and there are a lot more of those, hence reducing calculation times.

### Read industrial zones to geojson dict

In [0]:
industrial_zones = shapefile_to_geojson(industrial_zones_shape_path)

### Bounding boxes for images and industrial zones

In [0]:
def bounding_box(dataset, resolution=1000):
  src_crs, dst_crs = dataset.crs, rasterio.crs.CRS.from_epsg(4326)

  left, bottom, right, top = dataset.bounds
  hor, ver = numpy.linspace(left, right, resolution), numpy.linspace(bottom, top, resolution)
  coords = (
      list(zip(*rasterio.warp.transform(src_crs, dst_crs, hor, [bottom] * resolution))) +
      list(zip(*rasterio.warp.transform(src_crs, dst_crs, hor, [top] * resolution))) +
      list(zip(*rasterio.warp.transform(src_crs, dst_crs, [left] * resolution, ver))) + 
      list(zip(*rasterio.warp.transform(src_crs, dst_crs, [right] * resolution, ver)))
  )

  def min_max(x):
    return min(x), max(x)

  (lon0, lon1), (lat0, lat1) = map(min_max, zip(*coords))
  return (lon0, lat0, lon1, lat1)

In [0]:
sample_boxes = []

for sample in ProgressBar(samples):
  image_filename = f'{image_path}/{sample}.tif'
  with rasterio.open(image_filename) as dataset:
    sample_boxes.append(box(*bounding_box(dataset)))

In [0]:
def map_coords(coords, op=min):
  """Map `op` on a list of lists of ... lists of two floats, hence a
  polygon-like coordinate structure. Returns two floats."""
  if len(coords) == 2 and type(coords[0]) == float:
    return coords
  else:
    coords = [map_coords(coords, op) for coords in coords]
    return tuple(
        op(c[i] for c in coords)
        for i in [0, 1]
    )

In [0]:
industrial_zone_boxes = []

for feature in industrial_zones:
  x0, y0 = map_coords(feature['geometry']['coordinates'], min)
  x1, y1 = map_coords(feature['geometry']['coordinates'], max)

  industrial_zone_boxes.append(
    box(x0, y0, x1, y1)
  )

### Calculations

One or two minutes

In [0]:
for sample, sample_bbox in zip(ProgressBar(samples), sample_boxes):
  image_filename = f'{image_path}/{sample}.tif'
  
  with rasterio.open(image_filename) as dataset:
    mask_shape = (dataset.height, dataset.width)
    coordinate_to_pixel = ~dataset.transform
    src_crs, dst_crs = dataset.crs, rasterio.crs.CRS.from_epsg(4326)

  def transform(poly):
    if all(
        len(item) == 2 and all(
            isinstance(item, numbers.Number) for item in item)
        for item in poly
    ):
      return [
          (x, y)  * coordinate_to_pixel
          for x, y in zip(
              *rasterio.warp.transform(dst_crs, src_crs, *zip(*poly)))
      ]
    return list(map(transform, poly))
  
  mask = numpy.zeros(mask_shape, numpy.uint8)
  intersects = 0
  for feature, feature_bbox in zip(industrial_zones, industrial_zone_boxes):
    if not sample_bbox.intersects(feature_bbox):
      continue
    
    intersects += 1
    pixel_geometry = {
        'type': feature['geometry']['type'],
        'coordinates': transform(feature['geometry']['coordinates'])
    }
    
    for ext in iter_exteriors(shape(pixel_geometry)):
      xs, ys = map(numpy.array, ext.xy)
      pts = numpy.round(numpy.concatenate([
          xs[None, :, None],
          ys[None, :, None]
      ], axis=2)).astype(numpy.int64)

      cv2.fillPoly(mask, pts, 1)
  imsave(f'{industrial_zones_path}/{sample}.tif', mask)

### Examples

Nine random cutout for 5 random images.

In [0]:
for idx in numpy.random.choice(len(samples), 5, replace=False):
  sample = samples[idx]
  
  im = imread(f'{image_path}/{sample}.tif')
  mask = imread(f'{industrial_zones_path}/{sample}.tif')
  w = 256
  ys, xs = numpy.where((im > 0).min(axis=2) & (mask > 0))
  xs = numpy.clip(xs, w, mask.shape[1] - w)
  ys = numpy.clip(ys, w, mask.shape[0] - w)
  subsample = numpy.random.choice(len(ys), len(axes), replace=False)

  s = sorted(im[..., 1:4].ravel()[::100000])
  t = s[int(0.95 * len(s))]

  fig, axes = pyplot.subplots(5, 3, figsize=(23, 30))
  fig.suptitle(sample)
  axes = axes.ravel()

  for axis, x, y in zip(axes, xs[subsample], ys[subsample]):
    region = tuple([slice(i-w, i+w) for i in [y, x]])
    
    rgb = im[region][..., [3,2,1]] / t
    rgb = numpy.clip(rgb, 0, 1)
    axis.imshow(rgb)
    
    overlay = (mask[region][..., None] > 0).astype(numpy.float32)
    nothing = numpy.ones_like(overlay)
    overlay = numpy.dstack([overlay, 0*nothing, 0*nothing, 0.7*nothing])
    axis.imshow(overlay)

Output hidden; open in https://colab.research.google.com to view.

### Industry recall (and precision)

The assumption is that we are only intested in buildings in instrustial areas. Therefore it is insightfull to know whether there are building areas outside of the industrial zones (indicated by recall < 1) and how much of the industrial areas are labeled buildings.

In [0]:
# fp, tp, fn, tn = 0, 0, 0, 0
# for sample in ProgressBar(samples):
#   label = imread(f'{mask_path}/{sample}.tif') > 0
#   industry = imread(f'{industrial_zones_path}/{sample}.tif') > 0
#   fp += (~label &  industry).sum()
#   tp += ( label &  industry).sum()
#   fn += ( label & ~industry).sum()
#   tn += (~label & ~industry).sum()
  
fp, tp, fn, tn = 134046546, 7119565, 4166758, 7329411931

In [0]:
print(f'recall:    {tp / (tp + fn)}')
print(f'precision: {tp / (tp + fp)}')

recall:    0.6308135076410626
precision: 0.05043395294781479
